In [61]:
# import libraries:
import pandas as pd
import numpy as np
import random

In [63]:
# get inputs:
min_v = int(input("Enter the minimum value (min_v) you want to achieve: "))
max_w = int(input("Enter the maximum weight (max_w) allowed: "))

min_n = int(input("Enter the minimum number of snack types (min_n): "))
max_n = int(input("Enter the maximum number of snack types (max_n): "))


Enter the minimum value (min_v) you want to achieve:  12
Enter the maximum weight (max_w) allowed:  10
Enter the minimum number of snack types (min_n):  2
Enter the maximum number of snack types (max_n):  4


In [64]:
# read CSV file:
file_path = 'snacks.csv'
df = pd.read_csv(file_path)


In [65]:
print(df)

            Snack  Available Weight  Value
0          MazMaz                10     10
1   Doogh-e-Abali                15     10
2            Nani                 5      5
3            Jooj                 7     15
4         Hot-Dog                20     15
5           Chips                 8      6
6        Nooshaba                12      8
7        Shokolat                 6      7
8       Chocoroll                 9     12
9         Cookies                11     11
10        Abnabat                 4      4
11   Adams-Khersi                14      9
12        Popcorn                16     13
13         Pastil                 3      7
14       Tordilla                10      9
15       Masghati                 5      6
16        Ghottab                 7     10
17   Saghe-Talaei                 9     11
18    Choob-Shoor                13     12


In [66]:
#Part One: Defining Basic Concepts

#Gene:
class Gene:
    def __init__(self, name, max_weight, value_per_weight):
        self.name = name
        self.weight = 0
        self.value_per_weight = value_per_weight
        self.max_weight = max_weight
    def __repr__(self):
        return f"Gene(name={self.name}, weight={self.weight}, max_weight={self.max_weight}, value_per_weight={self.value_per_weight})\n"

    def __str__(self):
        return f"{self.name}: Weight={self.weight}, Value per Weight={self.value_per_weight}"

# Genes pool:
genes_pool = df.copy()
genes_pool.rename(columns={'Snack': 'Name'}, inplace=True)
genes_pool.rename(columns={'Available Weight': 'Maximum Weight'}, inplace=True)
genes_pool['Value per Weight'] = genes_pool['Value'] / genes_pool['Maximum Weight']
genes_pool = genes_pool[['Name', 'Maximum Weight', 'Value per Weight']]

gene_objects = [Gene(row['Name'], row['Maximum Weight'], row['Value per Weight']) for index, row in genes_pool.iterrows()]

#Chromosome:
class Chromosome:
    def __init__(self, genes_pool, x):
        self.genes = random.sample(genes_pool, x)  
        for gene in self.genes:
            gene.weight = random.uniform(0, gene.max_weight)
        self.total_weight = sum(gene.weight for gene in self.genes)
        self.total_value = sum(gene.weight * gene.value_per_weight for gene in self.genes)
        self.variety_of_snacks = len(self.genes)
        self.fitness = 0  
        
    def __repr__(self):
        return f"Chromosome(genes={self.genes}, fitness={self.fitness}) \n"

    def __str__(self):
        genes_str = '\n'.join(str(gene) for gene in self.genes)
        return f"Chromosome Details:\nGenes:\n{genes_str}\nFitness: {self.fitness} \nTotal Weight: {self.total_weight}\nTotal Value: { self.total_value}\nRange: {self.variety_of_snacks}\n"




In [ ]:
print(genes_pool)

In [ ]:
print(gene_objects)

In [69]:
#Part Two: Primary population production
population_size = 1000

initial_population = [Chromosome(gene_objects, random.randint(min_n, max_n)) for _ in range(population_size)]

In [ ]:
initial_population

In [87]:
#Part three: Implementation and specification of compatibility criterion function


def calculate_fitness(chromosome):
    fitness = chromosome.total_value
    penalty_weight =max_w - chromosome.total_weight
    penalty_value = chromosome.total_value - min_v
    #penalty_varity = min(chromosome.variety_of_snacks - max_n , min_n - chromosome.variety_of_snacks)
    penalties = 0
    if penalty_weight < 0:
        penalties = penalty_weight
    if penalty_value < 0:
        penalties = penalties + penalty_value
    if penalties < 0 :
        fitness = penalties
    return fitness
    


In [88]:
#Update Chromosomes Fitness'

for Chromosome in initial_population:
    Chromosome.fitness = calculate_fitness(Chromosome)

-12.815510655482335
-4.68930379342536
6.549638270590867
1.009280598936293
-5.752194592274986
-4.678903119915098
-15.981344779559656
-12.417537167520038
-4.9753237957124234
5.571665246839339
-0.8987580000806172
4.024682188016519
-7.741091129259878
-9.546447800924707
-4.770616200665112
-14.261553960285259
-5.557326999751179
-5.714920472643406
2.9800927270658937
0.312812822536225
-9.993467241525607
-15.164027961434773
-1.7506698220713695
-0.8533678813261112
-13.116273168855745
3.024967452718421
-1.2808349538471688
-11.43503009726684
-1.8304976836098472
6.291141685380998
-4.194701864021862
0.7333990549922103
7.345896508444248
-3.914262632479776
-3.0707034091296777
-20.398707228819486
-8.522343049758838
3.8364483541678815
-1.5372808951943142
4.783889717056062
-1.0268881777573782
0.353784564575367
2.237558935170032
8.99072398133945
-5.8897064472657945
6.081423852083126
0.4730403348692356
-3.5716491759677726
-8.143364226088462
-6.6193969951001925
-6.0402494749621525
-10.638741752852074
1.3965

In [89]:
initial_population

[Chromosome(genes=[Gene(name=Pastil, weight=2.1510600001718156, max_weight=3, value_per_weight=2.3333333333333335)
 , Gene(name=Jooj, weight=3.4818068472006396, max_weight=7, value_per_weight=2.142857142857143)
 , Gene(name=Nooshaba, weight=10.80544872798933, max_weight=12, value_per_weight=0.6666666666666666)
 , Gene(name=Saghe-Talaei, weight=4.663858922006666, max_weight=9, value_per_weight=1.2222222222222223)
 ], fitness=-12.815510655482335) ,
 Chromosome(genes=[Gene(name=Cookies, weight=5.765049162768821, max_weight=11, value_per_weight=1.0)
 , Gene(name=Ghottab, weight=6.200493471784219, max_weight=7, value_per_weight=1.4285714285714286)
 ], fitness=-4.68930379342536) ,
 Chromosome(genes=[Gene(name=Nani, weight=1.1370112645908685, max_weight=5, value_per_weight=1.0)
 , Gene(name=Jooj, weight=3.4818068472006396, max_weight=7, value_per_weight=2.142857142857143)
 ], fitness=-6.7931186066702205) ,
 Chromosome(genes=[Gene(name=Pastil, weight=2.1510600001718156, max_weight=3, value_per

In [90]:
def find_winner(population):
    max_fitness = float('-inf')
    winner = population[0];
    for Chromosome in population:
        if (Chromosome.fitness > max_fitness):
            max_fitness = Chromosome.fitness
            winner = Chromosome
    print(winner)
    return winner

In [91]:
def check_for_answer(population):
    winner = find_winner(population)
    if winner.fitness >= min_v:
        print(winner)
        return True
    print("not found!")
    return False


In [92]:
check_for_answer(initial_population)
print(calculate_fitness(find_winner(initial_population)))

Chromosome Details:
Genes:
Jooj: Weight=3.4818068472006396, Value per Weight=2.142857142857143
Chocoroll: Weight=4.184889521335102, Value per Weight=1.3333333333333333
Pastil: Weight=2.1510600001718156, Value per Weight=2.3333333333333335
Fitness: 16.710481956237366 
Total Weight: 9.827038331943301
Total Value: 16.710481956237366
Range: 3

Chromosome Details:
Genes:
Jooj: Weight=3.4818068472006396, Value per Weight=2.142857142857143
Chocoroll: Weight=4.184889521335102, Value per Weight=1.3333333333333333
Pastil: Weight=2.1510600001718156, Value per Weight=2.3333333333333335
Fitness: 16.710481956237366 
Total Weight: 9.827038331943301
Total Value: 16.710481956237366
Range: 3

Chromosome Details:
Genes:
Jooj: Weight=3.4818068472006396, Value per Weight=2.142857142857143
Chocoroll: Weight=4.184889521335102, Value per Weight=1.3333333333333333
Pastil: Weight=2.1510600001718156, Value per Weight=2.3333333333333335
Fitness: 16.710481956237366 
Total Weight: 9.827038331943301
Total Value: 16.

In [ ]:
#Part four: Generating a new generation

#